In [1]:
import requests
import pandas as pd

API_KEY = 'YOUR_API_KEY'  # Replace with your Alpha Vantage API key

def get_interest_rates():
    url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&apikey={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame.from_dict(data['data'], orient='columns')

def get_cpi_data():
    url = f'https://www.alphavantage.co/query?function=CPI&apikey={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame.from_dict(data['data'], orient='columns')

def get_gdp_data():
    url = f'https://www.alphavantage.co/query?function=REAL_GDP&apikey={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame.from_dict(data['data'], orient='columns')

def get_unemployment_data():
    url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey={API_KEY}'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame.from_dict(data['data'], orient='columns')

# Fetching data
interest_rates = get_interest_rates()
cpi_data = get_cpi_data()
gdp_data = get_gdp_data()
unemployment_data = get_unemployment_data()

# Print the dataframes
print("Interest Rates:")
print(interest_rates)

print("\nCPI Data:")
print(cpi_data)

print("\nGDP Data:")
print(gdp_data)

print("\nUnemployment Data:")
print(unemployment_data)


Interest Rates:
           date value
0    2024-08-01  5.33
1    2024-07-01  5.33
2    2024-06-01  5.33
3    2024-05-01  5.33
4    2024-04-01  5.33
..          ...   ...
837  1954-11-01  0.83
838  1954-10-01  0.85
839  1954-09-01  1.07
840  1954-08-01  1.22
841  1954-07-01  0.80

[842 rows x 2 columns]

CPI Data:
            date    value
0     2024-08-01  314.796
1     2024-07-01  314.540
2     2024-06-01  314.175
3     2024-05-01  314.069
4     2024-04-01  313.548
...          ...      ...
1335  1913-05-01    9.700
1336  1913-04-01    9.800
1337  1913-03-01    9.800
1338  1913-02-01    9.800
1339  1913-01-01    9.800

[1340 rows x 2 columns]

GDP Data:
          date      value
0   2023-01-01  22376.906
1   2022-01-01  21822.037
2   2021-01-01  21407.692
3   2020-01-01  20234.074
4   2019-01-01  20692.087
..         ...        ...
90  1933-01-01    877.431
91  1932-01-01    888.414
92  1931-01-01   1019.977
93  1930-01-01   1089.785
94  1929-01-01   1191.124

[95 rows x 2 columns]

U

In [2]:
import pandas as pd
import numpy as np

# Mock function to simulate fetching economic data
def get_economic_data():
    # Simulated data (you would replace this with actual API calls)
    data = {
        'date': pd.date_range(start='2020-01-01', periods=100),
        'interest_rate': np.random.uniform(0.01, 0.05, size=100),  # Simulated interest rates
        'inflation_rate': np.random.uniform(1, 3, size=100),  # Simulated inflation
        'gdp_growth_rate': np.random.uniform(1, 5, size=100),  # Simulated GDP growth
        'unemployment_rate': np.random.uniform(3, 10, size=100),  # Simulated unemployment
        'currency_price': np.random.uniform(1, 2, size=100)  # Simulated currency prices
    }
    return pd.DataFrame(data)

# Fetch economic data
data = get_economic_data()


In [3]:
pip install gym

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
import gym
from gym import spaces



In [5]:
pip install stable-baselines3


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\programdata\anaconda3\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [6]:
class ForexTradingEnv(gym.Env):
    def __init__(self, data):
        super(ForexTradingEnv, self).__init__()
        self.data = data.reset_index(drop=True)
        self.current_step = 0
        self.action_space = spaces.Discrete(3)  # Buy, Sell, Hold
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(5,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self.data.iloc[self.current_step][['interest_rate', 'inflation_rate', 'gdp_growth_rate', 'unemployment_rate', 'currency_price']].values.astype(np.float32)

    def step(self, action):
        # Check if we are at the last step before proceeding
        if self.current_step >= len(self.data) - 1:
            done = True
            return self.reset(), 0, done, {}

        # Increment step
        self.current_step += 1

        # Calculate reward based on action
        price_change = self.data.iloc[self.current_step]['currency_price'] - self.data.iloc[self.current_step - 1]['currency_price']
        reward = 0

        if action == 0:  # Buy
            reward = price_change
        elif action == 1:  # Sell
            reward = -price_change
        elif action == 2:  # Hold
            reward = 0

        # Get the next state
        next_state = self.data.iloc[self.current_step][['interest_rate', 'inflation_rate', 'gdp_growth_rate', 'unemployment_rate', 'currency_price']].values.astype(np.float32)
        done = self.current_step >= len(self.data) - 1  # Check if done

        return next_state, reward, done, {}


In [7]:
# Initialize the Forex trading environment
env = ForexTradingEnv(data)


In [8]:
df = pd.DataFrame(data)
    # Ensure numeric types
df['interest_rate'] = pd.to_numeric(df['interest_rate'])
df['inflation_rate'] = pd.to_numeric(df['inflation_rate'])
df['gdp_growth_rate'] = pd.to_numeric(df['gdp_growth_rate'])
df['unemployment_rate'] = pd.to_numeric(df['unemployment_rate'])
df['currency_price'] = pd.to_numeric(df['currency_price'])

In [9]:
from stable_baselines3 import PPO


In [10]:
# Create the agent
model = PPO("MlpPolicy", env, verbose=1)

# Train the agent with fewer timesteps for testing
model.learn(total_timesteps=1000)

# Save the model
model.save("forex_agent")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


C:\Users\cl501_27\AppData\Roaming\Python\Python311\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 99       |
|    ep_rew_mean     | -0.358   |
| time/              |          |
|    fps             | 1539     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------


In [11]:
# Load the model for testing
model = PPO.load("forex_agent")

# Test the agent
obs = env.reset()
total_reward = 0
for _ in range(len(data) - 1):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    total_reward += reward
    if done:
        break

print(f"Total Reward: {total_reward}")


Total Reward: 1.7296277413097236


In [12]:
import matplotlib.pyplot as plt

# Initialize the environment
obs = env.reset()
cumulative_reward = 0
rewards = []
prices = []

for _ in range(len(data) - 1):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    cumulative_reward += reward
    rewards.append(cumulative_reward)
    prices.append(env.data.iloc[env.current_step]['currency_price'])
    if done:
        break

# Plotting
plt.figure(figsize=(14, 7))

# Plot cumulative rewards
plt.subplot(2, 1, 1)
plt.plot(rewards, label='Cumulative Reward')
plt.title('Agent Cumulative Reward Over Time')
plt.xlabel('Steps')
plt.ylabel('Cumulative Reward')
plt.legend()

# Plot currency prices
plt.subplot(2, 1, 2)
plt.plot(prices, label='Currency Price', color='orange')
plt.title('Currency Price Over Time')
plt.xlabel('Steps')
plt.ylabel('Price')
plt.legend()

plt.tight_layout()
plt.show()


: 